## Import libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import itertools

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing

## Import data

In [2]:
#data retrive from https:lazyprogrammer.me/course_files/airline_passengers.csv
df = pd.read_csv('AirPassengers.csv',index_col = 'Month', parse_dates=True)

In [3]:
df.index.freq = 'MS'

In [4]:
# Checking the number of sample (row, column)
df.shape

(144, 1)

In [5]:
# Checking the number of sample
len(df)

144

In [6]:
df.head()

,#Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


## Working out with the walk forward cross valdiation

### Step 1: Indetify the number of train, validation and test size

In [7]:
# Assume the forecast horizon we care is about 12
# Validate over 10 steps
h = 12
steps = 10
Ntest = len(df) - h - steps + 1

In [8]:
Ntest

123

### Step 2: Set some configuration options

In [9]:
# Configurations hyperparameters to try
trend_type_list = ['add','mul']
seasonal_type_list = ['add','mul']
damped_trend_list = [True, False]
init_method_list = ['estimated','heuristic','legacy-heuristic']
use_boxcox_list = [True, False, 0]

### Step 3: Set function or walk forward cross validation

In [13]:
def walkforward(
    trend_type,
    seasonal_type,
    damped_trend,
    init_method,
    use_boxcox,
    debug=False):
    
    # store errors from each round
    errors = []
    seen_last = False # a flag that will tell us whether or not we've looped up to the final row in the df
    steps_completed = 0 # increment this by one on each round and by the end of the loop it should be equal to no of steps defined earlier
    
    for end_of_train in range(Ntest, len(df) - h  + 1):
        train = df.iloc[:end_of_train]
        test = df.iloc[end_of_train: end_of_train + h]
    
        if test.index[-1] == df.index[-1]:
            seen_last = True
    
        steps_completed += 1
    
        # Instantiate the model
        hw = ExponentialSmoothing(
            train['#Passengers'], 
            initialization_method = init_method,
            trend = trend_type,
            damped_trend = damped_trend,
            seasonal = seasonal_type,
            seasonal_periods=12,
            use_boxcox = use_boxcox)
        res_hw = hw.fit()
    
        # Compute error for the forecast horizon
        fcast = res_hw.forecast(h)
        error = mean_squared_error(test['#Passengers'], fcast)
        errors.append(error)
    
    if debug:
        print ("seen_last:", seen_last)
        print ("steps completed:", steps_completed)
        
    return np.mean(errors)

### Step 4: Test our function

In [14]:
walkforward (trend_type = 'add',
             seasonal_type = 'add',
             damped_trend = False,
             init_method = 'legacy-heuristic',
             use_boxcox = 0,
             debug = True)

seen_last: True
steps completed: 10


2521.0305142383577

### Step 5: Iterate through all possible functions (i.e. grid search)

In [16]:
tuple_of_option_lists = (trend_type_list,
                         seasonal_type_list,
                         damped_trend_list,
                         init_method_list,
                         use_boxcox_list)

for x in itertools.product(*tuple_of_option_lists):
    print(x)

('add', 'add', True, 'estimated', True)
('add', 'add', True, 'estimated', False)
('add', 'add', True, 'estimated', 0)
('add', 'add', True, 'heuristic', True)
('add', 'add', True, 'heuristic', False)
('add', 'add', True, 'heuristic', 0)
('add', 'add', True, 'legacy-heuristic', True)
('add', 'add', True, 'legacy-heuristic', False)
('add', 'add', True, 'legacy-heuristic', 0)
('add', 'add', False, 'estimated', True)
('add', 'add', False, 'estimated', False)
('add', 'add', False, 'estimated', 0)
('add', 'add', False, 'heuristic', True)
('add', 'add', False, 'heuristic', False)
('add', 'add', False, 'heuristic', 0)
('add', 'add', False, 'legacy-heuristic', True)
('add', 'add', False, 'legacy-heuristic', False)
('add', 'add', False, 'legacy-heuristic', 0)
('add', 'mul', True, 'estimated', True)
('add', 'mul', True, 'estimated', False)
('add', 'mul', True, 'estimated', 0)
('add', 'mul', True, 'heuristic', True)
('add', 'mul', True, 'heuristic', False)
('add', 'mul', True, 'heuristic', 0)
('add

In [18]:
best_score = float('inf')
best_options = None
for x in itertools.product(*tuple_of_option_lists):
    score = walkforward(*x)
    
    if score < best_score:
        print ("Best score so far:", score)
        best_score = score
        best_options = x

Best score so far: 412.81720294821025
Best score so far: 412.7068213361361
Best score so far: 320.6640771493606


C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarn

Best score so far: 303.71164998411547


C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarn

Best score so far: 263.33618316574746


C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarn

We get some warnings about overflows. This probably not an issue since it just means we'll get a bad model

### Step 6: Check the best scores

In [19]:
print ("best score:", best_score)

trend_type, seasonal_type, damped_trend, init_method, use_boxcox = best_options
print ("Trend type:", trend_type)
print ("Seasonal type:", seasonal_type)
print ("Damped trend:", damped_trend)
print ("Initialization method:", init_method)
print ("Use boxcox:", use_boxcox)

best score: 263.33618316574746
Trend type: mul
Seasonal type: mul
Damped trend: True
Initialization method: legacy-heuristic
Use boxcox: False


We can see that the best options for trend and seasonal type are multiplicative. This make sense since the seasonal pattern seems to grow over time for airline passengers